In [318]:
from django.db.models import Func
from django.db.models.functions import Rank, DenseRank, RowNumber, Cast
from django.db.models import F, Window, DurationField, ExpressionWrapper, DateTimeField, IntegerField


class Days(Func):
    """Cast float field to Interval in days"""
    output_field=DurationField()
    template="to_char(%(expressions)s, 'S9990.0999 \"days\"')::interval"
    
class Epoch(Func):
    """Get epoch timestamp from date time field """
    output_field=FloatField()
    template="extract(epoch from %(expressions)s) / 3600"

class Order(Func):
    output_field=FloatField()

adjusted_publication_time=ExpressionWrapper(F('story__publication_date') + Days('priority'), output_field=DateTimeField())

qs0 = FrontpageStory.objects.published()

qs1 = qs0\
    .annotate(rank=Window(expression=RowNumber(), order_by=adjusted_publication_time.desc()))\
    .annotate(order2=Epoch(adjusted_publication_time, output_field=IntegerField()))\
    .order_by('rank')

%timeit -n10 -r10 ll = list(qs1)
%timeit -n10 -r10 ll = list(qs0)

The slowest run took 495.80 times longer than the fastest. This could mean that an intermediate result is being cached.
11.3 ms ± 33.1 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
The slowest run took 255.65 times longer than the fastest. This could mean that an intermediate result is being cached.
6.26 ms ± 17.3 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [292]:
%timeit?